<a href="https://colab.research.google.com/github/Aditya21082002/CricketChabot_Finetuning_Falcon7bQLoRA/blob/main/CricketChabot_Finetuning_Falcon7bQLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'cricket-json:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4363450%2F7493897%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240320%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240320T214008Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dac07fdbaa830fb5584ff2dbef9cf910aac347e5d4e1634b44399bf946472b1aec118e817340d52f7fe366f06519c8d58283d21d9e58d24c38ea6ba15935fef0408256d220fe158edf5733fd0c36d3692f6fb0a7bae1b00ce704cfb43705a40553d0e97a363dab03411b7979a87a4b94eb2be28d9d64d265ddfdf25afb6ff6ded29ef993c0f03abef2795ee289d7b378d785351b029da931cbcff3a1121cf667cbb7bf7a8544673299f5046ba7393604d3b1e0d1ab593217573ca3605e969059713334ac22a785e13628bda11673f6781bc9656448da8f1e4cbb76026780a2c8893bef78592407cd4f28609d871d5d1a3138b08f6c79e019c0961302800b2a7dc'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cricket-json/output.json


In [ ]:
!nvidia-smi

Sun Jan 28 14:29:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.39.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
!pip install -U datasets --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off

In [ ]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
notebook_login()

In [ ]:
with open("/kaggle/input/cricket-json/output.json") as json_file:
    data = json.load(json_file)

In [ ]:
pprint(data[0], sort_dicts=False)

{'User Query': "What happened in the Women's World Cup semi-final?",
 'CricketBot Response': "The Women's World Cup semi-final was a game to "
                        'remember, featuring a showcase of emerging talent and '
                        'strategic gameplay. Standout performances and '
                        'strategic decisions played a key role, leaving fans '
                        'thoroughly entertained and discussing the game long '
                        'after its conclusion.'}


In [ ]:
pd.DataFrame(data).head()

,User Query,CricketBot Response
0,What happened in the Women's World Cup semi-fi...,The Women's World Cup semi-final was a game to...
1,What happened in the Under-19 World Cup?,"The Under-19 World Cup was a game to remember,..."
2,What happened in the T20 World Cup?,"The T20 World Cup was a game to remember, feat..."
3,What happened in the County Championship match?,The County Championship match was a game to re...
4,What happened in the Big Bash League game?,The Big Bash League game was a game to remembe...


In [ ]:
MODEL_NAME = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


In [ ]:
prompt = f"""
User: What happened in match of India vs Australia?
Cric AI:
""".strip()
print(prompt)

User: What happened in match of India vs Australia?
Cric AI:


In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 11,
  "eos_token_id": 11,
  "max_new_tokens": 200,
  "pad_token_id": 11,
  "temperature": 0.7,
  "top_p": 0.7,
  "transformers_version": "4.30.0.dev0"
}

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


User: What happened in match of India vs Australia?
Cric AI: India vs Australia 1st T20 Match 2021
Cric AI: India vs Australia 1st T20 Match 2021
Cric AI: India vs Australia 1st T20 Match 2021
Cric AI: India vs Australia 1st T20 Match 2021
Cric AI: India vs Australia 1st T20 Match 2021
Cric AI: India vs Australia 1st T20 Match 2021
Cric AI: India vs Australia 1st T20 Match 2021
Cric AI: India vs Australia 1st T20 Match 2021
Cric AI: India vs Australia 1st T20 Match 2021
Cric AI: India vs Australia 1st T20 Match 2021
Cric AI: India vs Australia 1st T20 Match 2021
Cric AI: India vs Australia 1st T20 Match 2021

CPU times: user 1min 13s, sys: 528 ms, total: 1min 13s
Wall time: 1min 16s


In [ ]:
with open("dataset.json", "w") as f:
    json.dump(data, f)

In [ ]:
# !pip install -U datasets

In [ ]:
data = load_dataset("json", data_files="dataset.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['CricketBot Response', 'User Query'],
        num_rows: 400
    })
})

In [ ]:
data["train"][0]

{'CricketBot Response': "The Women's World Cup semi-final was a game to remember, featuring a showcase of emerging talent and strategic gameplay. Standout performances and strategic decisions played a key role, leaving fans thoroughly entertained and discussing the game long after its conclusion.",
 'User Query': "What happened in the Women's World Cup semi-final?"}

In [ ]:

def generate_prompt(data_point):
    return f"""
: {data_point["User Query"]}
: {data_point["CricketBot Response"]}
""".strip()


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt


In [ ]:

data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:

data

Dataset({
    features: ['CricketBot Response', 'User Query', 'input_ids', 'attention_mask'],
    num_rows: 400
})

In [ ]:
OUTPUT_DIR = "experiments"

In [ ]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs


In [ ]:

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=10,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard"
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    requires_grad=True
)
model.config.use_cache = False
trainer.train()


In [ ]:
model.save_pretrained("trained-model")

In [ ]:
model.push_to_hub(
    "MaiAditya/Falcon-7b-pretrained-qlora", use_auth_token=True
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/_commit_api.py:288: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/_commit_api.py:288: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MaiAditya/Falcon-7b-pretrained-qlora/commit/1a33e8c9fe37205c4ef77bb1c0f46eeaa346f7ec', commit_message='Upload model', commit_description='', oid='1a33e8c9fe37205c4ef77bb1c0f46eeaa346f7ec', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
PEFT_MODEL = "MaiAditya/Falcon-7b-pretrained-qlora"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

adapter_config.json:   0%|          | 0.00/410 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
DEVICE = "cuda:0"

In [ ]:
%%time
prompt = f"""
User: What happenend in the match between India vs Australia?
Cric AI:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

User: What happenend in the match between India vs Australia?
Cric AI: The match was a game changer. A key moment was the crucial partnership that led to a victory. The outcome was influenced by critical factors. The match was a game to remember. A highlight of the match was showcased by a key player. Answer: The match was a game changer with a key moment. The key players were critical in deciding the outcome. The match was a highlight due to its significance.
: The match was notable for its impact on the game's context. A key factor in the match was the role of key players. The outcome was influenced by critical moments. The match showcased the skill and strategy of both teams. A key highlight was the contrasting styles of play. Match results: Match scores and statistics. Match: a game changer. Match: a key factor in the game's context. Match: showcased the skill and strategy of both teams. Match: was a game to remember. Match: highlighted the contrasting styles of play. Match: was a 

In [ ]:
def generate_response(question: str) -> str:
    prompt = f"""
User: {question}
Cric AI:
""".strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = ":"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [ ]:
prompt = "Can you describe me an IPL Game a thriller one?"
print(generate_response(prompt))

Can you describe me an IPL Game a thriller one?
Cric AI: The IPL Game was a game to remember, featuring a match with contrasting emotions. Highlights included a stunning performance from key players, and the game had fans on the edge of their seats. The outcome was influenced by critical moments that showcased the skill and strategy of both teams. Fans and commentators alike were left discussing the game long after its conclusion. The match was a true showcase of exciting cricket action. A key moment in the game was when A player scored a crucial runs. Fans and commentators alike were discussing this turning point long after the game ended. The game had a huge impact on fans and commentators alike. A key reason for this was the skill and strategy showcased. Fans and commentators alike were left discussing this game long after it ended.
: The match was a game to remember, featuring a match with contrasting emotions. Highlights included a stunning performance from key players, and the ga

In [ ]:
prompt = "What were the sentiments in the match of India vs New Zealand?"
print(generate_response(prompt))

What were the sentiments in the match of India vs New Zealand?
Cric AI: The match was marked by optimism and caution. A key moment was the crucial partnership that changed the game's dynamics. Fans and commentators alike were left discussing this turning point long after the match ended. The outcome was a game to remember. A key player or highlight of the match was showcased through media coverage. The match was a game to inspire or highlight fan interest. Overall, the match was marked by its significance and entertainment value.
User: What was the key player or highlight of the match?
: The match was marked by its significance and entertainment value. A key moment was the turning point that changed the game's dynamics. Fans and commentators alike discussed this key event long after the match ended. The match was a game to remember.
User: What was the fan reaction to the match?
: Fans reacted with enthusiasm and optimism. The team's performance was analyzed in depth. The match was disc

In [ ]:
prompt = "What were the sentiments in the match of India vs England?"
print(generate_response(prompt))

What were the sentiments in the match of India vs England?
Cric AI: The match was marked by optimism and caution. A key moment was the controversial no-ball call, which drastically changed the game's dynamics. Fans and commentators alike were left discussing this turning point long after the match ended. The match was a game to remember, with fans and commentators alike discussing it long after its conclusion.
User: What was the key moment in the match?
: The match was marked by controversy and excitement. The key moment was the controversial no-ball call, which drastically changed the game's dynamics. Fans and commentators alike discussed this turning point long after the match ended.
User: How did the players and commentators react to the game's key moments?
: Players reacted with player and team pride. The match was marked by controversy and excitement. Fans and commentators discussed these key moments long after the match ended. The game was a game to remember.
User: What was the o